# Neural architecture search for Multi-Layer Perceptron

### First defining the parameters to create a dictionary file containing a primary key for all possible combination of parameters


In [1]:
from parameters import *

In [2]:
from parameters import *

def vocab_dict():
    vocab = {}
    
    for i in range(len(nodes)):
        for j in range(len(activations)):
            
            # Calculate layer ID
            layer_id = len(activations) * i + j + 1
            
            # Store layer parameters in the vocabulary dictionary
            vocab[layer_id] = (nodes[i], activations[j])
    
    # Add dropout information to the vocabulary
    vocab[len(vocab) + 1] = (('dropout'))
    
    if target_classes == 2:
        vocab[len(vocab) + 1] = (target_classes - 1, 'sigmoid')
    else:
        vocab[len(vocab) + 1] = (target_classes, 'softmax')
    return vocab

vocab = vocab_dict()
print('vocab size : ' ,len(vocab))

final_vocab_length = len(vocab)

# The final id of the vocabulary will always be the size of output classes and a softmax layer

print('Final layer id : ',vocab[final_vocab_length])

# the semi-final layer 
print("Second last layer of the vocabulary : ",vocab[final_vocab_length-1])

vocab size :  23
Final layer id :  (3, 'softmax')
Second last layer of the vocabulary :  dropout


In [3]:
## Step 2: sample architecture from the vocabulary according to the size of the defined model

final_layer_id = len(vocab)
dropout_id = final_layer_id -1
print('final_layer_id',final_layer_id)
print('dropout_id:',dropout_id)

vocab_idx = [0]+list(vocab.keys())
print('vocab ids:',vocab_idx)

final_layer_id 23
dropout_id: 22
vocab ids: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [5]:
from keras.preprocessing.sequence import pad_sequences

samples = []
seed = []

sequence = pad_sequences([seed],maxlen = 10-1,padding = 'post')
print(sequence)

2023-11-08 08:03:47.750908: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 08:03:47.828572: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 08:03:47.832339: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-08 08:03:49.702581: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[[0 0 0 0 0 0 0 0 0]]


In [6]:
sequence = sequence.reshape(1,1,9)
print(sequence)

[[[0 0 0 0 0 0 0 0 0]]]


In [7]:
# ## Define a LSTM model to choose between sequences?
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# 
# model = keras.Sequential()
# model.add(layers.LSTM(controller_input_shape, input_shape=(None, 28)))
# model.add(layers.BatchNormalization())
# model.add(layers.Dense(10))
# print(model.summary())

In [8]:
controller_classes = len(vocab) + 1
print(controller_classes)

24


In [9]:
from keras.layers import Input,LSTM,Dense
from keras.models import Model

main_input = Input(
                    shape = controller_input_shape, 
                    batch_size = controller_batch_size, 
                    name = 'main_input')

x = LSTM(controller_lstm_dim, 
         return_sequences = True)(main_input)

main_output = Dense(controller_classes, 
                    activation = 'softmax', 
                    name = 'main_output')(x)

model = Model(inputs = [main_input], 
              outputs = [main_output])

print(model.summary())

2023-11-08 08:03:55.592884: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-08 08:03:55.593760: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 main_input (InputLayer)     [(32, 1, 9)]              0         
                                                                 
 lstm (LSTM)                 (32, 1, 100)              44000     
                                                                 
 main_output (Dense)         (32, 1, 24)               2424      
                                                                 
Total params: 46424 (181.34 KB)
Trainable params: 46424 (181.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [31]:
sequence = sequence.reshape(1,1,max_len-1)
print(sequence.shape)

probab = model.predict(sequence)
print(probab)

(1, 1, 9)
1/1 [==============================] - 0s 27ms/step
[[[0.04166667 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667
   0.04166667 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667
   0.04166667 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667
   0.04166667 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667]]]


In [32]:
probab = probab[0][0]
print(probab)

[0.04166667 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667
 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667
 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667
 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667]


In [35]:
import numpy as np
next = np.random.choice(vocab_idx,size=1, p = probab)[0]
print(next)

21


In [36]:
seed.append(next)
print(seed)

[21]


In [37]:
sequence = pad_sequences([seed],maxlen=max_len-1,padding='post')
print(sequence)

[[21  0  0  0  0  0  0  0  0]]


In [40]:
sequence = sequence.reshape(1, 1, max_len - 1)
print(sequence)

[[[21  0  0  0  0  0  0  0  0]]]


In [46]:
probab = model.predict(sequence)
print(probab)

1/1 [==============================] - 0s 29ms/step
[[[0.0273195  0.04185842 0.04114107 0.03899648 0.03210942 0.03029226
   0.0248489  0.04742223 0.02984473 0.04737589 0.01995707 0.08189879
   0.02961613 0.03817816 0.04517929 0.03946372 0.07375851 0.07337391
   0.04744702 0.05357071 0.04997896 0.01759756 0.02853585 0.0402353 ]]]


In [47]:
next = np.random.choice(vocab_idx,size=1, p=probab)

ValueError: 'p' must be 1-dimensional